In [ ]:
tensorflow_version 1.x"

In [ ]:
# images and labels were saved as numpy files in my private drive so I mount it so I can use it later
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#imports
from keras.models import model_from_json
import numpy as np
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Conv3D, GlobalAveragePooling2D
from keras import backend as K
from keras import optimizers
from keras import layers
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from collections import Counter
from sklearn.metrics import confusion_matrix



In [ ]:
#load data, print data shape to check database size& each class size 
images=np.load('/content/drive/My Drive/Final_project_breast_cancer/mammos.npy')
labels=np.load('/content/drive/My Drive/Final_project_breast_cancer/labels.npy')
print(images.shape)
print(labels.shape)

class0=labels[:,0]
class1=labels[:,1]
sum0=sum(class0)
sum1=sum(class1)
print('num of objects in class 0:',sum0,'\n','num of objects in class 1:', sum1)

In [ ]:
#filter data to seperate class0 &class data
"""filtering data - class separation"""
def filter(data,labels):
  x_c0=[]
  x_c1=[]
  y_c0=[]
  y_c1=[]
  for i in range(0,len(data)):
    if labels[i,0]==1:
      x_c0.append(data[i,:,:,:])
      y_c0.append(labels[i,:])
    else:
      x_c1.append(data[i,:,:,:])
      y_c1.append(labels[i,:])
  return x_c0,y_c0,x_c1,y_c1

class0_data, class0_labels, class1_data, class1_labels=filter(images,labels)
class0_data=np.array(class0_data)
class0_labels=np.array(class0_labels)
class1_data=np.array(class1_data)
class1_labels=np.array(class1_labels)
#validate filtering
print('class0 data shape:', class0_data.shape)
print('class0 labels shape:',class0_labels.shape)
print('class1 data shape:',class1_data.shape)
print('class1 labels shape:',class1_labels.shape)

In [ ]:
# Get test and val data befor augmenting and get train data after undersampling to balance classes
test_data=[*class0_data[:200,:,:,:],*class1_data[:200,:,:,:]]
test_label=[*class0_labels[:200,:], *class1_labels[:200,:]]
test_data=np.array(test_data)
test_label=np.array(test_label)

s = np.arange(test_data.shape[0])
np.random.shuffle(s)
test_data=test_data[s]
test_label=test_label[s]

val_data=[*class0_data[200:400,:,:,:],*class1_data[200:400,:,:,:]]
val_label=[*class0_labels[200:400,:],*class1_labels[200:400,:]]
val_data=np.array(val_data)
val_label=np.array(val_label)
s = np.arange(val_data.shape[0])
np.random.shuffle(s)
val_data=val_data[s]
val_label=val_label[s]
print('test_data:' ,test_data.shape)
print('test_labels',test_label.shape)

print('val data:',val_data.shape)
print('val labels:',val_label.shape)

class0_data=class0_data[400:927,:,:,:]
class0_labels=class0_labels[400:927,:]
class1_data=class1_data[400:927,:,:,:]
class1_labels=class1_labels[400:927,:]

del images,labels,class0,class1

In [ ]:
# Augmentaion parameters

# Define wanted augmentation
datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.7, 1.3),
    rotation_range=15
    
    )

In [ ]:
#class 1 augmentation

# Fit on class1
datagen.fit(class1_data)

# class1 augmentation
class1_aug_data=[]
class1_aug_labels=[]
for i in range (0,10):
  for x_batch, y_batch in datagen.flow(class1_data, class1_labels, batch_size=500):
    break
  class1_aug_data.append(x_batch)
  class1_aug_labels.append(y_batch)
class1_aug_data=np.array(class1_aug_data)
class1_aug_labels=np.array(class1_aug_labels)


class1_aug_data=class1_aug_data.reshape(5000,200,200,1)
class1_aug_labels=class1_aug_labels.reshape(5000,2)

#merge augmentation + original data
class1_total_data=[*class1_data,*class1_aug_data]
class1_total_labels=[*class1_labels,*class1_aug_labels]
class1_total_data=np.array(class1_total_data)
class1_total_labels=np.array(class1_total_labels)
print('class1 objects&shape after resampling:',class1_total_data.shape)
print('class1 label shape after resampling;' ,class1_total_labels.shape)
print('\n class1 images:')

#print to vmake sure images were not damaged
for j in range(0, 4):
  plt.subplot(2,2,1 + j)
  plt.imshow(class1_total_data[j].reshape(200, 200), cmap='gray')
  plt.show()

In [ ]:
del class1_data
del class1_aug_data
del class1_labels
del class1_aug_labels

In [ ]:

#class0 augmentation
class0_aug_data=[]
class0_aug_labels=[]
datagen.fit(class0_data)

for i in range(0,10):
  for x_batch, y_batch in datagen.flow(class0_data, class0_labels, batch_size=500):
    break
  class0_aug_data.append(x_batch)
  class0_aug_labels.append(y_batch)

class0_aug_data=np.array(class0_aug_data)
class0_aug_labels=np.array(class0_aug_labels)


class0_aug_data=class0_aug_data.reshape(5000,200,200,1)
class0_aug_labels=class0_aug_labels.reshape(5000,2)

#merge augmentation+original data
class0_total_data=[*class0_data,*class0_aug_data]
class0_total_labels=[*class0_labels,*class0_aug_labels]
class0_total_data=np.array(class0_total_data)
class0_total_labels=np.array(class0_total_labels)

print('class0 objects&shape after resampling:',class0_total_data.shape)
print('class0 label shape after resampling;' ,class0_total_labels.shape)

#print some images
print('\n class0 images:')
for j in range(0, 4):
  plt.subplot(2,2,1 + j)
  plt.imshow(class0_aug_data[j].reshape(200, 200), cmap='gray')
  plt.show()



In [ ]:
del x_batch
del y_batch

In [ ]:
del class0_data
del class0_aug_data
del class0_labels
del class0_aug_labels

In [ ]:
#combine data to train set
train_data=[*class0_total_data ,*class1_total_data]
train_labels=[*class0_total_labels, *class1_total_labels]
train_data=np.array(train_data)
train_labels=np.array(train_labels)

print(train_data.shape)
print(train_labels.shape)

In [ ]:
del class0_total_data
del class0_total_labels
del class1_total_data
del class1_total_labels

In [ ]:
#shuffle train set and print shape + amount of data in each class (to make sure amount is expected and balanced)
s = np.arange(train_data.shape[0])
np.random.shuffle(s)
train_data=train_data[s]
train_labels=train_labels[s]

train_labels=np.array(train_labels)
train_data=np.array(train_data)
print('balanced data shape:', train_data.shape)
print('combined label shape:', train_labels.shape)

cls0=train_labels[:,0]
cls1=train_labels[:,1]
sum0=sum(cls0)
sum1=sum(cls1)
print('\n','num of objects in class 0:',sum0)
print(' num of objects in class 1:', sum1)

print(train_labels)



In [ ]:
#validate images are intact in all sets
print('data examples')

train_im=train_data[1298,:,:,:].reshape(200,200)
val_im=val_data[190,:,:,:].reshape(200,200)
test_im=test_data[380,:,:,:].reshape(200,200)

plt.subplot(131)
plt.imshow(train_im, cmap='gray')
plt.title('Train image')
plt.subplot(132)
plt.imshow(val_im, cmap='gray')
plt.title('val image')
plt.subplot(133)
plt.imshow(test_im, cmap='gray')
plt.title('Test image')

In [ ]:
#model parameters

batch_size = 32
num_classes = 2
epochs = 35
beta_1=0.9
beta_2=0.999
epsilon=10e-8
reg_value= 1e-3

# input image dimensions
img_rows, img_cols = 200, 200

train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

In [ ]:
#this block is Transfer learning network imports. to prevent importing all networks each time use case to choose which network to import
case=2
#look at cases for desired net
##############################################
if (case==1):
  from keras.applications import vgg16
  vgg16=vgg16.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(200,200,3)
  )
############################################# 
if (case==2):
  from keras.applications import densenet
  dense201=densenet.DenseNet201(
    weights='imagenet',
    include_top=False,
    input_shape=(200,200,3)
    
  ) 
  dense169=densenet.DenseNet169(
    weights='imagenet',
    include_top=False,
    input_shape=(200,200,3)
  )
  dense121=densenet.DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(200,200,3)
  ) 
###############################################
if case==3:
  from keras.applications import inception_v3
  V3=inception_v3.InceptionV3(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  )
###########################################
if case==4:
  from keras.applications import xception
  XCEPT=xception.Xception(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
)
##########################################
if case ==5:
  from keras.applications import vgg19
  vgg19=vgg19.VGG19(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  )
#####################################
if case==6:
  from keras.applications import resnet
  res50=resnet.ResNet50(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  )
  res101=resnet.ResNet101(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  )
  res152=resnet.ResNet152(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  )
#########################################
if case==7:
  from keras.applications import resnet_v2
  res50V2=resnet_v2.ResNet50V2(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  )
  res101V2=resnet_v2.ResNet101V2(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  )
  res152V2=resnet_v2.ResNet152V2(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  ) 
#######################################
if case==8:
  from keras.applications import inception_resnet_v2
  V2=inception_resnet_v2.InceptionResNetV2(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  )
#######################################
if case==9:
  from keras.applications import mobilenet
  mobile=mobilenet.MobileNet(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  )
###################################
if case==10:
  from keras.applications import mobilenet_v2
  mobile=mobilenet_v2.MobileNetV2(
      weights='imagenet',
      include_top=False,
      input_shape=(200,200,3)
  )
######################################
if case==11:
  from keras.applications import nasnet
  nas_l=nasnet.NASNetLarge(
        weights=None,
        include_top=False,
        input_shape=(200,200,3)
    )
  
  nas_m=nasnet.NASNetMobile(
        weights=None,
        include_top=False,
        input_shape=(200,200,3)
    )



In [ ]:
# this block decides which layers in TL network will train
print("Number of layers in the base model: ", len(dense201.layers))
for layer in dense201.layers:
  layers.trainable = False
for layer in dense201.layers[358:]:
  layer.trainable =  True
for layer in dense201.layers[350:362]:
  print(layer.trainable)




In [ ]:
lr=8e-5
from keras.layers import AveragePooling2D
adam = optimizers.Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon)
# callbacks - stopping stops while val acc does not increase after 7 iter. reduceLR lowers Learning rate if val loss is not decreasing
stopping=keras.callbacks.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.005, patience=7, mode='max', baseline=0.7,restore_best_weights=True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=7,
                      min_delta=0.005, mode='min', verbose=1)
callbacks_list = [reduceLROnPlat, stopping]

#model architecture
def build_model(backbone, lr=8e-5):
    model = Sequential()
    model.add(Conv2D(3, (1,1), padding='same', kernel_initializer='ones',trainable=False))
    model.add(backbone)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.6))
    model.add(layers.BatchNormalization())
    model.add(Dense(8, kernel_regularizer=regularizers.l1(reg_value*10), activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(4, activation='relu',trainable=False))
    model.add(Dropout(0.6))
    model.add(layers.Dense(2, kernel_regularizer=regularizers.l1(reg_value*10), activation='softmax'))
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=adam,
        metrics=['accuracy']
    )
    return model

# choose TL network (I used dense201)
model = build_model(dense201,lr = 8e-5)

#run training
history = model.fit(train_data, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks_list,
         validation_data = (val_data, val_label))

#print train&val loss graph
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show(); plt.close()

In [ ]:
#print model architecture+ num of parameters
model.summary()


In [ ]:
# confusion matrix function
import itertools
import numpy as np
import matplotlib.pyplot as plt


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        normed_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        normed_cm = normed_cm*100;
        print("Normalized confusion matrix")
        print(normed_cm)
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.figure(figsize=(15,15))
    plt.imshow(normed_cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i, format(normed_cm[i, j], fmt)+'%\n('+(format(cm[i, j], 'd'))+')',
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.gcf().subplots_adjust(bottom=0.3)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#print validation set confusion matrix
from sklearn.metrics import confusion_matrix
val_pred = model.predict(val_data)

# Confusion Matrix
cm = confusion_matrix(np.argmax(val_label,axis=1), np.argmax(np.round(val_pred),axis=1));
labels = ['class ' + str(i) for i in range(num_classes)] 
plot_confusion_matrix(cm,labels,title='Confusion Matrix',normalize=True)

In [ ]:
#print test score
#####################################################
score = model.evaluate(test_data, test_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
####################################################

In [ ]:
#print test confusion matrix
from sklearn.metrics import confusion_matrix
test_pred = model.predict(test_data)

# Confusion Matrix
cm = confusion_matrix(np.argmax(test_label,axis=1), np.argmax(np.round(test_pred),axis=1));
labels = ['class ' + str(i) for i in range(num_classes)] 
plot_confusion_matrix(cm,labels,title='Confusion Matrix',normalize=True)

In [ ]:
#Data for ROC
test_prob=model.predict_proba(test_data)
y_t=test_label[:,0]
y_p=test_prob[:,0]
print('y_t shape:', y_t.shape)
print('y_p shape:', y_p.shape)

print('y_train shape:', y_t.shape)
print('y_test shape:', y_p.shape)

In [ ]:
#print ROC curve
from sklearn.metrics import roc_curve,roc_auc_score
[fpr,tpr,thresholds] = roc_curve ( y_t , y_p)

def plot_roc_curve(fpr,tpr): 
  plt.plot(fpr,tpr,'--') 
  plt.axis([0,1,0,1]) 
  plt.xlabel('False Positive Rate') 
  plt.ylabel('True Positive Rate') 
  plt.show()    
  
plot_roc_curve (fpr,tpr)

In [ ]:
#AUC calc
from sklearn.metrics import roc_auc_score
roc_auc_score(y_t,y_p)